## Deliverable 2. Create a Customer Travel Destinations Map.
This notebook uses user input and the Google Maps API to create an upto date hotel recommendation using the Weather_Database.csv file.
 - My weather settings are currently set to Minimum Temperature 75F, Maximum Temperature 90F, and No rain.
 - Google Maps API info retrieval happens @ cell 8 and hotel Database saved in cell 9.
 - If you do not mind using my weather settings and do not want to want to pull the current hotel recomendations from the Google Maps API, start at cell 10 where the recommended hotel Database is loaded from csv.
 - Google maps API key is still required to create and view the map of recommended cities with hotels. 


In [73]:
# Python Built-in Modules
import os
import requests
import time

# Third Party Modules
import pandas as pd
import numpy as np
import gmaps

# Local Files
from config import gkey

# filepath for the weather data created so far
cities_filepath = os.path.join('..','Weather_Database', 'WeatherPy_DataBase.csv')

# configure Google maps to use the api key
gmaps.configure(api_key=gkey)

In [74]:
# View the data
city_data = pd.read_csv(cities_filepath).drop('Unnamed: 0', axis=1)
city_data

,City,Country,Date,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Neiafu,TO,2022-07-23 02:38:56,-18.6500,-173.9833,79.48,73.0,40.0,5.75,scattered clouds
1,Tecoanapa,MX,2022-07-23 02:43:40,16.5167,-98.7500,80.87,81.0,38.0,3.58,scattered clouds
2,Aasiaat,GL,2022-07-23 02:43:43,68.7098,-52.8699,39.29,99.0,100.0,5.17,light rain
3,Pangai,TO,2022-07-23 02:43:45,-19.8000,-174.3500,78.91,78.0,40.0,13.80,scattered clouds
4,Klaksvik,FO,2022-07-23 02:43:47,62.2266,-6.5890,49.12,82.0,100.0,6.46,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
2165,Iskateley,RU,2022-07-23 04:41:15,67.6803,53.1512,54.12,90.0,99.0,10.45,overcast clouds
2166,Dunyapur,PK,2022-07-23 04:41:17,29.7995,71.7196,93.63,54.0,100.0,7.09,overcast clouds
2167,San ramon,BO,2022-07-23 04:37:05,37.7799,-121.9780,67.75,74.0,0.0,8.05,clear sky
2168,Vydrino,RU,2022-07-23 04:41:21,51.4850,104.8461,55.17,89.0,100.0,1.63,overcast clouds


In [53]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum Temperature for your trip?  "))
max_temp = float(input("What is the maximum Temperature for your trip?  "))
rain = input("Do you like rain? (Yes/No)  ")

What is the minimum Temperature for your trip?  75
What is the maximum Temperature for your trip?  90
Do you like rain? (Yes/No)  NO


In [54]:
#  Filter the city_data_df DataFrame using the input statements to create a new DataFrame.
# The rain question filters for places that are more likely to rain or not. 

if 'yes' in rain.lower():
    preferred_cities = city_data[(city_data['Max Temp']>=min_temp)&(city_data['Max Temp']<=max_temp)&((city_data['Description'].str.contains('rain')==True) | (city_data['Cloudiness']>=50))]
elif 'no' in rain.lower():
    preferred_cities = city_data[(city_data['Max Temp']>=min_temp)&(city_data['Max Temp']<=max_temp)&(city_data['Description'].str.contains('rain')==False) & (city_data['Cloudiness']<=50)]
else:     
    preferred_cities = city_data[(city_data['Max Temp']>=min_temp)&(city_data['Max Temp']<=max_temp)]

preferred_cities


,City,Country,Date,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Neiafu,TO,2022-07-23 02:38:56,-18.6500,-173.9833,79.48,73.0,40.0,5.75,scattered clouds
1,Tecoanapa,MX,2022-07-23 02:43:40,16.5167,-98.7500,80.87,81.0,38.0,3.58,scattered clouds
3,Pangai,TO,2022-07-23 02:43:45,-19.8000,-174.3500,78.91,78.0,40.0,13.80,scattered clouds
7,Vaini,TO,2022-07-23 02:43:54,-21.2000,-175.2000,75.36,83.0,40.0,12.66,scattered clouds
11,Hakui,JP,2022-07-23 02:44:01,36.8833,136.7833,79.45,81.0,38.0,8.57,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
2126,Hohhot,CN,2022-07-23 04:38:57,40.8106,111.6522,79.09,47.0,0.0,6.71,clear sky
2128,Vanimo,PG,2022-07-23 04:39:00,-2.6741,141.3028,84.51,72.0,20.0,8.77,few clouds
2143,Udine,IT,2022-07-23 04:40:29,46.0619,13.2422,77.81,67.0,0.0,2.30,clear sky
2152,San javier,BO,2022-07-23 04:40:48,37.8063,-0.8374,77.13,84.0,7.0,4.61,clear sky


In [55]:
# find hotels in the preffered cities
api_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [56]:
hotel_df = preferred_cities.reset_index()[['City', 'Country', 'Lat', 'Lon', 'Max Temp', 'Description']].copy()
hotel_df['Hotel Name'] = ''

# Hotels are 'lodging', within 5000 meters, location needs  to be a string  "lat, long"
payload = {'radius': 5000, 'key': gkey, 'types' : 'lodging'}  

# Counter for the number of cities that return an HTTP 404 Error
err_counter = 0

for i in hotel_df.index:
    payload['location'] = f"{hotel_df['Lat'][i]}, {hotel_df['Lon'][i]}"
    response = requests.get(api_url, params=payload).json()
 
    try: 
        hotel_df['Hotel Name'][i] = response['results'][0]['name']
    except KeyError:
        err_counter +=1
    except IndexError:
        err_counter +=1
    if (i % 150 == 0):
        # Progress Tracker and managing API call limit
        print(f'index: {i}  ||  HTTP 404 Errors so far: {err_counter} = {((err_counter/hotel_df.shape[0])*100):.2f}%') 
        time.sleep(60)
    elif (i % 50 == 0):
        # Just the Progress Tracker 
        print(f'index: {i}  ||  HTTP 404 Errors so far: {err_counter} = {((err_counter/hotel_df.shape[0])*100):.2f}%') 
    else:
        pass     

        
hotel_df        

C:\Users\Nick\AppData\Local\Temp\ipykernel_18424\3645457894.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'][i] = response['results'][0]['name']


index: 0  ||  HTTP 404 Errors so far: 0 = 0.00%
index: 50  ||  HTTP 404 Errors so far: 2 = 0.49%
index: 100  ||  HTTP 404 Errors so far: 7 = 1.72%
index: 150  ||  HTTP 404 Errors so far: 7 = 1.72%
index: 200  ||  HTTP 404 Errors so far: 7 = 1.72%
index: 250  ||  HTTP 404 Errors so far: 8 = 1.97%
index: 300  ||  HTTP 404 Errors so far: 12 = 2.95%
index: 350  ||  HTTP 404 Errors so far: 13 = 3.19%
index: 400  ||  HTTP 404 Errors so far: 18 = 4.42%


,City,Country,Lat,Lon,Max Temp,Description,Hotel Name
0,Neiafu,TO,-18.6500,-173.9833,79.48,scattered clouds,Mystic Sands Beachfront Accommodation
1,Tecoanapa,MX,16.5167,-98.7500,80.87,scattered clouds,Hotel Manglares
2,Pangai,TO,-19.8000,-174.3500,78.91,scattered clouds,Ha'apai Beach Resort
3,Vaini,TO,-21.2000,-175.2000,75.36,scattered clouds,Keleti Beach Resort
4,Hakui,JP,36.8833,136.7833,79.45,scattered clouds,KYUKAMURA NOTO-CHIRIHAMA
...,...,...,...,...,...,...,...
402,Hohhot,CN,40.8106,111.6522,79.09,clear sky,Sheraton Hohhot Hotel
403,Vanimo,PG,-2.6741,141.3028,84.51,few clouds,Vahmoneh Lodge Vanimo
404,Udine,IT,46.0619,13.2422,77.81,clear sky,Best Western Hotel Continental
405,San javier,BO,37.8063,-0.8374,77.13,clear sky,Hotel Mar Menor


In [59]:
# drop the cities that returned no Hotel results
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace('', np.nan)
hotel_df = hotel_df.dropna().reset_index().drop('index', axis=1)

# Save the hotel dataframe to a csv
hotel_filepath = os.path.join('WeatherPy_Vacation.csv')
hotel_df.to_csv(hotel_filepath, index_label='City_ID')


In [75]:
hotel_filepath = os.path.join('WeatherPy_Vacation.csv')
hotel_df = pd.read_csv(hotel_filepath, index_col='City_ID')
hotel_df

,City,Country,Lat,Lon,Max Temp,Description,Hotel Name
City_ID,,,,,,,
0,Neiafu,TO,-18.6500,-173.9833,79.48,scattered clouds,Mystic Sands Beachfront Accommodation
1,Tecoanapa,MX,16.5167,-98.7500,80.87,scattered clouds,Hotel Manglares
2,Pangai,TO,-19.8000,-174.3500,78.91,scattered clouds,Ha'apai Beach Resort
3,Vaini,TO,-21.2000,-175.2000,75.36,scattered clouds,Keleti Beach Resort
4,Hakui,JP,36.8833,136.7833,79.45,scattered clouds,KYUKAMURA NOTO-CHIRIHAMA
...,...,...,...,...,...,...,...
384,Hohhot,CN,40.8106,111.6522,79.09,clear sky,Sheraton Hohhot Hotel
385,Vanimo,PG,-2.6741,141.3028,84.51,few clouds,Vahmoneh Lodge Vanimo
386,Udine,IT,46.0619,13.2422,77.81,clear sky,Best Western Hotel Continental


In [76]:
# Create a heatmap for the Max Temp's with the new hotel_df
hotel_fig = gmaps.figure(center=(30,31), zoom_level=1.5)
temp_heatmap = gmaps.heatmap_layer(hotel_df[['Lat','Lon']], weights=hotel_df['Max Temp'], dissipating=False, max_intensity=350, point_radius=4)
hotel_fig.add_layer(temp_heatmap)

# add city markers and format them to have pop-up info: city name, country, hotel name, and Current Weather Description.
# Note: Google Maps' pop-up markers use html coded dictionaries: 
#  "term" : "definition"  ==  dt: dd  ==  <dt></dt> <dd></dd> 

info_box_template = """
<dl>
<dt>Hotel Name</dt>   <dd>{Hotel Name}</dd>
<dt>City</dt>   <dd>{City}</dd>
<dt>Country</dt>   <dd>{Country}</dd>
<dt>Current Weather</dt>   <dd>{Description} and {Max Temp}\u00B0F</dd>
</dl>
"""

# list comprehension that formats the city pop-up marker, added .format() here for iteration
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# adding city markers to the map
city_marker = gmaps.marker_layer(hotel_df[['Lat', 'Lon']], info_box_content=hotel_info)
hotel_fig.add_layer(city_marker)
hotel_fig



Figure(layout=FigureLayout(height='420px'))